In [26]:
import pandas as pd
import numpy as np
from gurobipy import *
import random
import itertools


In [27]:

# Parameters
teams = ["Adelaide Crows", 
         "Brisbane Lions", 
         "Carlton Blues", 
         "Collingwood Magpies",
         "Essendon Bombers", 
         "Fremantle Dockers", 
         "Geelong Cats", 
         "Gold Coast Suns",
         "Greater Western Sydney Giants", 
         "Hawthorn Hawks", 
         "Melbourne Demons",
         "North Melbourne Kangaroos", 
         "Port Adelaide Power",
         "Richmond Tigers",
         "St Kilda Saints",
         "Sydney Swans",
         "West Coast Eagles",
         "Western Bulldogs"]

team_numbers = {team: number for number, team in enumerate(sorted(teams), start=0)}

locations = ['VIC','NSW','SA','WA','QLD']

location_numbers = {location: number for number, location in enumerate(sorted(locations), start=0)}

home_locations = ['SA','QLD','VIC','VIC','VIC','WA','VIC','QLD','NSW','VIC',
                          'VIC','VIC','SA','VIC','VIC','NSW','WA','VIC']

ranking = [10,2,5,1,11,14,12,15,7,16,4,17,3,13,6,8,18,9] # 2023 pre-finals rankings

team_fans = [60,45,93,102,79,61,81,20,33,72,68,50,60,100,58,63,100,55] # 2023 number of members, in 000's

stadiums = ['MCG','Marvel','GMHBA','Adelaide Oval','Optus','Gabba','HBS','SCG','Giants']

stadium_numbers = {stadium: number for number, stadium in enumerate(sorted(stadiums), start=0)}

stadium_locations = ['VIC','VIC','VIC','SA','WA','QLD','QLD','NSW','NSW']

home_stadiums = [['Adelaide Oval'],['Gabba'],['MCG','Marvel'],['MCG','Marvel'],['MCG','Marvel'],['Optus'],['GMHBA'],
                 ['HBS'],['Giants'],['MCG'],['MCG'],['Marvel'],['Adelaide Oval'],
                 ['MCG'],['Marvel'],['SCG'],['Optus'],['Marvel']]

home_location_stadiums = [[] for i in range(len(teams))]
for i in range(len(teams)):
    for j in range(len(stadiums)):
        if stadium_locations[j] == home_locations[i]:
            home_location_stadiums[i].append(stadiums[j])

stadium_size = [100,53,40,54,60,39,27,47,24] # Stadium sizes in 000's

rivals = [['Port Adelaide Power'],
          ['Gold Coast Suns','Collingwood Magpies'], # From wikipedia
          ['Essendon Bombers','Richmond Tigers','Collingwood Magpies', 'Fremantle Dockers'],
          ['Carlton Blues','Essendon Bombers','Brisbane Lions','Melbourne Demons','Richmond Tigers','Geelong Cats',
           'Hawthorn Hawks', 'West Coast Eagles','Port Adelaide Power'],
          ['Carlton Blues','Collingwood Magpies','Richmond Tigers','Hawthorn Hawks','North Melbourne Kangaroos'],
          ['Carlton Blues','West Coast Eagles'],
          ['Collingwood Magpies','Hawthorn Hawks'],
          ['Brisbane Lions'],
          ['Sydney Swans','Western Bulldogs'],
          ['Hawthorn Hawks','Geelong Cats','Essendon Bombers', 'North Melbourne Kangaroos'],
          ['Collingwood Magpies'],
          ['Essendon Bombers','Hawthorn Hawks'],
          ['Adelaide Crows','Collingwood Magpies'],
          ['Carlton Blues','Collingwood Magpies','Essendon Bombers'],
          ['Sydney Swans'],
          ['West Coast Eagles','St Kilda Saints','Greater Western Sydney Giants','Hawthorn Hawks'],
          ['Fremantle Dockers','Sydney Swans','Collingwood Magpies'],
          ['Greater Western Sydney Giants']]

rivals_num = [[team_numbers[i] for i in rivals[j]] for j in range(len(rivals))]

timeslots = [i for i in range(7)]
timeslot_values = [100,130,50,60,110,50,40] # Change later according to attendances
timeslot_names = ['Thursday Night','Friday Night','Saturday Afternoon','Saturday Evening',
                  'Saturday Night','Sunday Afternoon','Sunday Evening']

rounds = [i for i in range(22)]

Ts = range(len(teams))
Ss = range(len(stadiums))
timeslots = range(7)
rounds = range(22)

# Decision Variables
fixture_matrix = [[[[[0 for i in Ts] for j in Ts] for s in Ss] for t in timeslots] for r in rounds]

In [40]:
# Attractiveness Function Parameters, adjust them as needed
alpha = 1.0
beta = 1.0
gamma = 1.0
sigma = 1.0
xi = 1.0

def attractiveness(i, j, s, t, r):
    score = 1
    
    if j in rivals[i]:
        score *= 1+alpha
    
    score /= max(abs(ranking[i]-ranking[j]),1)
    score  /= (ranking[i]+ranking[j])/2
    
    if stadium_locations[s] == home_locations[j]:
        score *= (1+beta)
        
    score *= stadium_size[s]
    score *= (team_fans[i]+team_fans[j])
    
    score *= timeslot_values[t]
    
    return score

In [41]:
def feasibility(fixture):
    violated = 0
    critical = 0
    
    # Each team plays once a week
    tally = 0
    for i in Ts: 
        for r in rounds:
            tally += sum(sum(sum(fixture[r][t][s][j][i] + fixture[r][t][s][i][j] for j in Ts) for s in Ss) for t in timeslots) 
    if tally != 1:
        critical += 1
            
    # Each team has eleven home games
    tally = 0
    for i in Ts:
        sum(sum(sum(sum(fixture[r][t][stadium_numbers[s]][j][i] for j in Ts) for s in home_stadiums[i]) for t in timeslots) for r in rounds) 
    if tally != 11:
        critical += 1
    
    # Teams can't play themselves, and play all other teams once or twice (not twice away, or twice home)
    tally1 = 0
    tally2 = 0
    tally3 = 0
    for i in Ts:
        tally += sum(sum(sum(fixture[r][t][s][i][i] for s in Ss) for t in timeslots) for r in rounds) 
        for j in Ts:
            if i != j:
                # Cannot play more than twice 
                tally2 += sum(sum(sum(fixture[r][t][s][j][i] for s in Ss) for t in timeslots) for r in rounds) 
                # Play at least once 
                tally3 += sum(sum(sum(fixture[r][t][s][j][i] + fixture[r][t][s][i][j] for s in Ss) for t in timeslots) for r in rounds) 
    if tally1 > 0:
        critical += 1      
    if tally2 > 1:
        critical += 1
    if tally3 < 1:
        critical += 1
    
    # At least a five day break 
    tally = 0        
    for i in Ts:
        for r in rounds[:-1]:
            tally = tally + sum(sum(sum(fixture[r][t][s][j][i]+fixture[r][t][s][i][j] for j in Ts) for s in Ss) for t in [5,6]) + sum(sum(sum(fixture[r+1][t][s][j][i]+fixture[r+1][t][s][j][i] for j in Ts) for s in Ss) for t in [0])
    if 1 < tally:
        violated += 1

    # No three games in a row outside home location
    tally = 0
    for i in Ts:
        for r in rounds[:-2]:
            tally += sum(sum(sum(sum(fixture[r_][t][stadium_numbers[s]][j][i]+fixture[r_][t][stadium_numbers[s]][i][j] for j in Ts) for s in home_location_stadiums[i]) for t in timeslots) for r_ in range(r,r+3)) 
    if tally < 1:
        violated += 1 

    # No four away games in a row
    tally = 0
    for i in Ts:
        for r in rounds[:-3]:
            tally += sum(sum(sum(sum(fixture[r_][t][s][j][i] for j in Ts) for s in Ss) for t in timeslots)
                                     for r_ in range(r,r+4)) 
    if tally < 1:
        violated += 1  

    # No 2+ games in one day in the same stadium
    for r in rounds:
        tally1 = 0
        tally2 = 0
        tally3 = 0
        for s in Ss:
            # Saturday & Sunday Games 
            tally1 += sum(sum(sum(fixture[r][t][s][j][i] for i in Ts) for j in Ts) for t in [5, 6]) 
            tally2 += sum(sum(sum(fixture[r][t][s][j][i] for i in Ts) for j in Ts) for t in [2, 3, 4])
            # One game per timeslot per stadium 
            for t in [0,1]:
                tally3 += sum(sum(fixture[r][t][s][j][i] for i in Ts) for j in Ts)  
        if tally1 > 1:
            violated += 1
        if tally2 > 1:
            violated += 1
        if tally3 > 1:
            violated += 1
    
    tally1 = 0
    tally2 = 0
    tally3 = 0
    tally4 = 0

    # No more than two games in any timeslot, and only one on Thursday and Friday night, incentivise games in each timeslot
    for r in rounds:
        
        # At least 2 sunday games 
        tally1 += sum(sum(sum(sum(fixture[r][t][s][j][i] for i in Ts) for j in Ts) for s in Ss) for t in [5,6])
        
        # One Thursday & Friday Night Game
        for t in [0,1]:
            tally2 += sum(sum(sum(fixture[r][t][s][j][i] for i in Ts) for j in Ts) for s in Ss)
        
        # No More Than Two Simultaneous Games
        for t in [2,3,4,5,6]:
            tally3 += sum(sum(sum(fixture[r][t][s][j][i] for i in Ts) for j in Ts) for s in Ss)     

        for t in timeslots:
            if sum(sum(sum(fixture[r][t][s][j][i] for i in Ts) for j in Ts) for s in Ss) < 1: 
                tally4 += 1
    
    if tally1 < 2:
        violated += 1
    if tally2 > 1: 
        violated += 1
    if tally3 > 2:
        violated += 1
    if tally4 > 0:
        violated += 1

    print(critical, violated)
    return violated, critical

In [42]:
def objective_value(fixture):

    violated, critical = feasibility(fixture)

    constraint_penalty = 1000
    critical_penalty = 1000

    sum(sum(sum(sum(sum(attractiveness(i,j,s,t,r) for i in Ts) for j in Ts) for s in Ss) for t in timeslots) for r in rounds)
    # for i in Ts:
    #     for j in Ts:
    #         for s in Ss:
    #             for r in rounds:
    #                 for t in timeslots:
    #                     print(attractiveness(i,j,s,t,r))

    objective_value = constraint_penalty*violated + critical_penalty*critical + sum(sum(sum(sum(sum(attractiveness(i,j,s,t,r) for i in Ts) for j in Ts) for s in Ss) for t in timeslots) for r in rounds) 

    return objective_value

In [43]:
def genesis(pop_size=1):
    pop = []
    for i in range(pop_size):

        fixture_matrix = [[[[[0 for i in Ts] for j in Ts] for s in Ss] for t in timeslots] for r in rounds]

        for r in rounds: 
            for t in timeslots:
                # Randomly choose two teams. Ensure there is at least one rivalry match. 
                if t == 0:
                    selection_i = random.randint(0, len(teams)-1)
                    selection_j = random.randint(0, len(rivals[selection_i])-1)
                    selection_j = team_numbers[rivals[selection_i][selection_j]]
                    selection_stadium = random.randint(0, len(home_location_stadiums[selection_i])-1)
                    fixture_matrix[r][t][selection_stadium][selection_j][selection_i] = 1 
                    # print('Rivalry Round: ', teams[selection_i], ' vs. ', teams[selection_j])
                else: 
                    selection_i = random.randint(0, len(teams)-1)
                    selection_j = random.randint(0, len(teams)-1)
                    if selection_j == selection_i:
                        selection_j -= 1
                    selection_stadium = random.randint(0, len(home_location_stadiums[selection_i])-1)
                    fixture_matrix[r][t][selection_stadium][selection_j][selection_i] = 1 
                    # print(teams[selection_i], ' vs. ', teams[selection_j])          
            # print('\n')  
        pop.append(fixture_matrix)
    return pop

In [44]:
pop = genesis()
individual = pop[0]
objective_value(individual)

3 69


117930513.9848972